# 🇹🇷 BIST 100 Portfolio Optimizer\n## Complete Optimization + Analysis\n\n**Features**: Data Download | Portfolio Optimization | Performance Analysis | Interactive Visualizations

## 📦 Setup

In [ ]:
#!pip install quantstats yfinance plotly pandas matplotlib seaborn scipy nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.2-py3-none-any.whl (24 kB)

   -------------------- ------------------- 1/2 [nbformat]
   ---------------------------------------- 2/2 [nbformat]



In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import quantstats as qs
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
qs.extend_pandas()
print('✅ Setup complete!')

✅ Setup complete!


## 🎯 Step 1: Define BIST 100 Universe

In [3]:
BIST100_TICKERS = [
    'AEFES.IS', 'AKBNK.IS', 'AKSA.IS', 'AKSEN.IS', 'ALARK.IS',
    'ANSGR.IS', 'ARCLK.IS', 'ASELS.IS', 'BIMAS.IS', 'BISAS.IS',
    'BRSAN.IS', 'BRYAT.IS', 'BTCIM.IS', 'BUCIM.IS', 'CCOLA.IS',
    'CIMSA.IS', 'CLEBI.IS', 'DOAS.IS', 'DOHOL.IS', 'ECILC.IS',
    'EGEEN.IS', 'EKGYO.IS', 'ENKAI.IS', 'EREGL.IS', 'FENER.IS',
    'FROTO.IS', 'GARAN.IS', 'GSRAY.IS', 'GUBRF.IS', 'GENTS.IS',
    'HALKB.IS', 'HEKTS.IS', 'IPDRO.IS', 'ISCTR.IS', 'ISFIN.IS',
    'KCHOL.IS', 'KOZAA.IS', 'KOZAL.IS', 'KRDMD.IS', 'MGROS.IS',
    'OYAKC.IS', 'MAKIM.IS', 'OTKAR.IS', 'PETKM.IS', 'SAHOL.IS',
    'SASA.IS', 'SISE.IS', 'SKBNK.IS', 'TAVHL.IS', 'TCELL.IS',
    'THYAO.IS', 'TKFEN.IS', 'TOASO.IS', 'TSKB.IS', 'TSPOR.IS',
    'TTKOM.IS', 'TTRAK.IS', 'TUKAS.IS', 'TUPRS.IS', 'ULKER.IS',
    'VAKBN.IS', 'VESTL.IS', 'AGHOL.IS', 'YKBNK.IS', 'ZOREN.IS',
    'KUYUM.IS', 'PGSUS.IS', 'ODAS.IS', 'MAVI.IS', 'ENJSA.IS',
    'MPARK.IS', 'SOKM.IS', 'KONTR.IS', 'TUREX.IS', 'CANTE.IS',
    'GENIL.IS', 'GRSLT.IS', 'YESIL.IS', 'MRGNE.IS', 'MIATK.IS',
    'DGKLY.IS', 'GRSEL.IS', 'KCAER.IS', 'ASTOR.IS', 'EUPWR.IS',
    'GRTHO.IS', 'CWENE.IS', 'KLVNE.IS', 'PAMEL.IS', 'ENJYO.IS',
    'REEDR.IS', 'TABGD.IS', 'BINHO.IS', 'PASEU.IS', 'OBAMS.IS',
    'ALTIN.IS', 'EFORC.IS', 'GLRMK.IS', 'DSFAK.IS', 'BALSU.IS'
]
print(f'📊 {len(BIST100_TICKERS)} stocks defined')

📊 100 stocks defined


## 📥 Step 2: Download Historical Data

In [4]:
# Configure date range
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1 year (changed from 730)

print(f'📅 Period: {start_date.date()} to {end_date.date()}')

📅 Period: 2025-01-01 to 2026-01-01


In [5]:
# Download data with progress
print('Downloading BIST 100 data...')
data = yf.download(BIST100_TICKERS, start=start_date, end=end_date, 
                   group_by='ticker', threads=True, progress=True)
print('✓ Download complete!')

[**********************47%                       ]  47 of 100 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DSFAK.IS"}}}
$GRSLT.IS: possibly delisted; no timezone found
[**********************53%                       ]  53 of 100 completed$DSFAK.IS: possibly delisted; no timezone found
[**********************54%*                      ]  54 of 100 completed$DGKLY.IS: possibly delisted; no timezone found
[**********************57%**                     ]  57 of 100 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KLVNE.IS"}}}
[**********************61%****                   ]  61 of 100 completed$KUYUM.IS: possibly delisted; no timezone found
[**********************69%********               ]  69 of 100 completed$KLVNE.IS: possibly delisted; no timezone found
[**********************71%*********              ]  71 of 100 completed$

✓ Download complete!


In [6]:
# Diagnostic: Check data structure
print('Data type:', type(data))
print('Data shape:', data.shape)
print('\nColumn structure:')
if isinstance(data.columns, pd.MultiIndex):
    print('MultiIndex columns - Levels:', data.columns.nlevels)
    print('Level 0 (first 5):', data.columns.levels[0][:5].tolist() if len(data.columns.levels[0]) > 0 else 'empty')
    print('Level 1:', data.columns.levels[1].tolist() if data.columns.nlevels > 1 else 'N/A')
else:
    print('Single level columns:', data.columns.tolist()[:10])
print('\nFirst few rows:')
data.head()

Data type: <class 'pandas.core.frame.DataFrame'>
Data shape: (252, 510)

Column structure:
MultiIndex columns - Levels: 2
Level 0 (first 5): ['AEFES.IS', 'AGHOL.IS', 'AKBNK.IS', 'AKSA.IS', 'AKSEN.IS']
Level 1: ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

First few rows:


Ticker     ZOREN.IS                                CCOLA.IS             \
Price          Open  High   Low Close     Volume       Open       High   
Date                                                                     
2025-01-02     4.37  4.43  4.33  4.43   58204781  58.705504  59.097198   
2025-01-03     4.43  4.49  4.39  4.49   85985251  57.579374  58.215883   
2025-01-06     4.50  4.66  4.49  4.63  171063540  57.922110  58.558615   
2025-01-07     4.63  4.64  4.51  4.52   85101612  57.726262  57.726262   
2025-01-08     4.51  4.53  4.37  4.38  102659505  55.522971  56.893906   

Ticker                                     ...   EFORC.IS             \
Price             Low      Close   Volume  ...       Open       High   
Date                                       ...                         
2025-01-02  57.089754  57.383526  5473497  ...   9.701938   9.875778   
2025-01-03  57.383526  57.873146  5060610  ...   9.768163  10.157234   
2025-01-06  57.138714  57.726261  4875490  ...  10.157234  10.372463   
2025-01-07  55.229198  55.229198  5382930  ...  10.264849  10.546304   
2025-01-08  55.327122  55.914665  5295826  ...  10.413854  10.471801   

Ticker                                          PGSUS.IS              \
Price             Low      Close      Volume        Open        High   
Date                                                                   
2025-01-02   9.652269   9.743329   9938705.0  214.100006  215.600006   
2025-01-03   9.643991  10.157234  12520181.0  216.899994  225.000000   
2025-01-06   9.834387  10.264849  15671693.0  224.100006  225.000000   
2025-01-07  10.231737  10.413855  12505733.0  221.600006  222.899994   
2025-01-08  10.206902  10.331074   8750819.0  218.500000  223.000000   

Ticker                                        
Price              Low       Close    Volume  
Date                                          
2025-01-02  212.800003  215.600006   5864562  
2025-01-03  216.199997  223.699997  13989064  
2025-01-06  222.000000  222.000000   6615557  
2025-01-07  218.600006  218.600006   7081388  
2025-01-08  217.500000  218.800003   6376797  

[5 rows x 510 columns]

In [7]:
# Extract adjusted close prices (improved extraction logic)
prices = pd.DataFrame()

# Check if data is empty
if data.empty:
    print('❌ No data downloaded - all tickers may have failed')
else:
    # Handle MultiIndex columns (group_by='ticker')
    if isinstance(data.columns, pd.MultiIndex):
        print(f'Processing MultiIndex data with {data.columns.nlevels} levels...')
        # Get unique tickers from level 0
        tickers_in_data = data.columns.get_level_values(0).unique()
        print(f'Found {len(tickers_in_data)} tickers in data')
        
        for ticker in tickers_in_data:
            try:
                # Try to get Adj Close column
                if (ticker, 'Adj Close') in data.columns:
                    prices[ticker] = data[(ticker, 'Adj Close')]
                elif (ticker, 'Close') in data.columns:
                    prices[ticker] = data[(ticker, 'Close')]
            except Exception as e:
                print(f'Skipping {ticker}: {e}')
    
    # Handle single-level columns
    else:
        print('Processing single-level columns...')
        if 'Adj Close' in data.columns:
            prices = data[['Adj Close']].copy()
        elif 'Close' in data.columns:
            prices = data[['Close']].copy()

print(f'\n✓ Extracted {len(prices.columns)} stocks')
print(f'✓ Rows: {len(prices)}')

# Clean data (relaxed criteria: 60% completeness + 20-day forward-fill)
if len(prices.columns) > 0:
    min_data = len(prices) * 0.60
    prices = prices[prices.columns[prices.count() >= min_data]]
    prices = prices.fillna(method='ffill', limit=20).dropna(axis=1)
    tickers = prices.columns.tolist()
    
    if len(tickers) > 0:
        print(f'\n✓ {len(tickers)} stocks with sufficient data')
        print(f'✓ Period: {prices.index[0].date()} to {prices.index[-1].date()}')
        print(f'✓ Trading days: {len(prices)}')
    else:
        print('\n⚠️ No stocks passed the data quality filters')
else:
    print('\n❌ No price data extracted')
    tickers = []

Processing MultiIndex data with 2 levels...
Found 100 tickers in data

✓ Extracted 100 stocks
✓ Rows: 252

✓ 88 stocks with sufficient data
✓ Period: 2025-01-02 to 2025-12-31
✓ Trading days: 252


In [8]:
# Preview data
prices.tail()

,ZOREN.IS,CCOLA.IS,EUPWR.IS,KOZAA.IS,CLEBI.IS,GSRAY.IS,KONTR.IS,AKSA.IS,KCHOL.IS,BUCIM.IS,...,TSKB.IS,OYAKC.IS,FENER.IS,GENIL.IS,CWENE.IS,HEKTS.IS,MPARK.IS,GRSEL.IS,EFORC.IS,PGSUS.IS
Date,,,,,,,,,,,,,,,,,,,,,
2025-12-25,3.05,60.299999,31.480000,107.699997,1554.0,1.20,11.40,9.97,171.000000,7.25,...,12.71,23.799999,9.43,182.000000,29.000000,3.14,378.75,323.5,26.639999,206.500000
2025-12-26,3.05,60.049999,33.580002,107.599998,1542.0,1.19,11.45,9.83,168.500000,7.32,...,12.50,23.160000,9.54,186.000000,29.000000,3.08,379.00,324.0,25.100000,204.500000
2025-12-29,3.02,58.049999,32.119999,103.599998,1515.0,1.16,11.33,9.71,167.399994,7.31,...,12.27,23.120001,9.34,187.500000,28.360001,3.05,372.75,317.5,26.400000,195.399994
2025-12-30,3.07,58.799999,33.720001,103.900002,1518.0,1.15,10.80,9.93,168.699997,7.55,...,12.28,22.860001,9.46,189.899994,27.959999,3.06,372.00,319.5,23.760000,192.199997
2025-12-31,3.07,58.799999,32.799999,104.400002,1523.0,1.16,11.52,9.82,168.800003,8.00,...,12.35,23.040001,9.22,199.000000,28.040001,3.06,380.50,320.0,24.200001,191.699997


## 🎯 Step 3: Portfolio Optimization

In [9]:
# Calculate returns
returns = prices.pct_change().dropna()

# Expected returns (annualized)
expected_returns = returns.mean() * 252

# Covariance matrix (annualized)
cov_matrix = returns.cov() * 252

# Risk-free rate (Turkish 1-year bond ~30%)
risk_free_rate = 0.30

print(f'✓ Returns calculated for {len(tickers)} stocks')
print(f'✓ Risk-free rate: {risk_free_rate*100:.1f}%')

✓ Returns calculated for 88 stocks
✓ Risk-free rate: 30.0%


In [10]:
# Optimization helper functions
def portfolio_stats(weights, returns, cov):
    ret = np.sum(returns * weights)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    return ret, vol

def neg_sharpe(weights, returns, cov, rf):
    ret, vol = portfolio_stats(weights, returns, cov)
    return -(ret - rf) / vol

def minimize_vol(weights, returns, cov):
    return portfolio_stats(weights, returns, cov)[1]

print('✓ Optimization functions defined')

✓ Optimization functions defined


In [11]:
# Run optimizations
n_assets = len(tickers)
init = np.array([1/n_assets] * n_assets)
cons = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
bnds = tuple((0, 0.15) for _ in range(n_assets))

print('Running optimizations...')

# Max Sharpe
max_sharpe = minimize(neg_sharpe, init, args=(expected_returns, cov_matrix, risk_free_rate),
                      method='SLSQP', bounds=bnds, constraints=cons)
ms_ret, ms_vol = portfolio_stats(max_sharpe.x, expected_returns, cov_matrix)
ms_sharpe = (ms_ret - risk_free_rate) / ms_vol

# Min Volatility (removed risk_free_rate from args - not needed)
min_vol = minimize(minimize_vol, init, args=(expected_returns, cov_matrix),
                   method='SLSQP', bounds=bnds, constraints=cons)
mv_ret, mv_vol = portfolio_stats(min_vol.x, expected_returns, cov_matrix)
mv_sharpe = (mv_ret - risk_free_rate) / mv_vol

print('✓ Optimizations complete')

Running optimizations...
✓ Optimizations complete


## 📊 Step 4: Optimization Results

In [12]:
# Summary table
results = pd.DataFrame({
    'Strategy': ['Maximum Sharpe', 'Minimum Volatility'],
    'Return (%)': [ms_ret*100, mv_ret*100],
    'Volatility (%)': [ms_vol*100, mv_vol*100],
    'Sharpe Ratio': [ms_sharpe, mv_sharpe],
    'Positions': [
        (max_sharpe.x > 0.001).sum(),
        (min_vol.x > 0.001).sum()
    ]
})

results

,Strategy,Return (%),Volatility (%),Sharpe Ratio,Positions
0,Maximum Sharpe,103.456612,22.856686,3.213791,13
1,Minimum Volatility,33.803945,17.790309,0.213821,27


In [13]:
# Extract top positions
max_sharpe_weights = pd.Series(max_sharpe.x, index=tickers)
max_sharpe_top = max_sharpe_weights[max_sharpe_weights > 0.01].sort_values(ascending=False)

min_vol_weights = pd.Series(min_vol.x, index=tickers)
min_vol_top = min_vol_weights[min_vol_weights > 0.01].sort_values(ascending=False)

print('MAX SHARPE TOP POSITIONS:')
print(max_sharpe_top.head(10))
print('\nMIN VOLATILITY TOP POSITIONS:')
print(min_vol_top.head(10))

MAX SHARPE TOP POSITIONS:
EFORC.IS    0.150000
ASELS.IS    0.150000
PASEU.IS    0.150000
HALKB.IS    0.123139
ECILC.IS    0.082179
GENTS.IS    0.078948
AKSEN.IS    0.070916
GRTHO.IS    0.059358
GENIL.IS    0.035948
ENKAI.IS    0.033664
dtype: float64

MIN VOLATILITY TOP POSITIONS:
TTRAK.IS    0.150000
TUPRS.IS    0.106118
TABGD.IS    0.092152
MPARK.IS    0.075389
PASEU.IS    0.073302
ANSGR.IS    0.064467
EFORC.IS    0.059921
OBAMS.IS    0.045293
DOHOL.IS    0.036586
GENIL.IS    0.035765
dtype: float64


In [14]:
# Visualize optimal portfolios
fig = go.Figure()

fig.add_trace(go.Bar(
    x=max_sharpe_top.index[:15],
    y=max_sharpe_top.values[:15]*100,
    name='Max Sharpe',
    marker_color='green'
))

fig.update_layout(
    title='Maximum Sharpe Ratio Portfolio - Top 15 Holdings',
    xaxis_title='Ticker',
    yaxis_title='Weight (%)',
    height=500
)

fig.show()

In [15]:
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=min_vol_top.index[:15],
    y=min_vol_top.values[:15]*100,
    name='Min Volatility',
    marker_color='blue'
))

fig2.update_layout(
    title='Minimum Volatility Portfolio - Top 15 Holdings',
    xaxis_title='Ticker',
    yaxis_title='Weight (%)',
    height=500
)

fig2.show()

## 📈 Step 5: Efficient Frontier

In [16]:
# Calculate efficient frontier
target_returns = np.linspace(expected_returns.min(), expected_returns.max(), 50)
frontier_vol = []
frontier_ret = []

for target in target_returns:
    cons_ef = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: portfolio_stats(x, expected_returns, cov_matrix)[0] - target}
    ]
    # Removed risk_free_rate from args - minimize_vol doesn't need it
    result = minimize(minimize_vol, init, args=(expected_returns, cov_matrix),
                     method='SLSQP', bounds=bnds, constraints=cons_ef)
    if result.success:
        ret, vol = portfolio_stats(result.x, expected_returns, cov_matrix)
        frontier_ret.append(ret)
        frontier_vol.append(vol)

print(f'✓ Efficient frontier calculated ({len(frontier_ret)} points)')

✓ Efficient frontier calculated (34 points)


In [17]:
# Plot efficient frontier
fig = go.Figure()

# Frontier
fig.add_trace(go.Scatter(
    x=np.array(frontier_vol)*100,
    y=np.array(frontier_ret)*100,
    mode='lines',
    name='Efficient Frontier',
    line=dict(color='blue', width=3)
))

# Max Sharpe
fig.add_trace(go.Scatter(
    x=[ms_vol*100],
    y=[ms_ret*100],
    mode='markers',
    name=f'Max Sharpe ({ms_sharpe:.2f})',
    marker=dict(size=15, color='green', symbol='star')
))

# Min Vol
fig.add_trace(go.Scatter(
    x=[mv_vol*100],
    y=[mv_ret*100],
    mode='markers',
    name=f'Min Volatility',
    marker=dict(size=15, color='red', symbol='diamond')
))

fig.update_layout(
    title='BIST 100 Efficient Frontier',
    xaxis_title='Volatility (%)',
    yaxis_title='Expected Return (%)',
    height=600,
    hovermode='closest'
)

fig.show()

## 🎲 Step 6: Monte Carlo Simulation

In [20]:
# Monte Carlo simulation
np.random.seed(42)
num_portfolios = 100000
mc_returns = np.zeros(num_portfolios)
mc_volatility = np.zeros(num_portfolios)
mc_sharpe = np.zeros(num_portfolios)

for i in range(num_portfolios):
    weights = np.random.random(n_assets)
    weights /= np.sum(weights)
    ret, vol = portfolio_stats(weights, expected_returns, cov_matrix)
    mc_returns[i] = ret
    mc_volatility[i] = vol
    mc_sharpe[i] = (ret - risk_free_rate) / vol

print(f'✓ {num_portfolios:,} random portfolios simulated')

✓ 100,000 random portfolios simulated


In [21]:
# Plot Monte Carlo results
fig = go.Figure()

# Random portfolios
fig.add_trace(go.Scatter(
    x=mc_volatility*100,
    y=mc_returns*100,
    mode='markers',
    name='Random Portfolios',
    marker=dict(
        size=3,
        color=mc_sharpe,
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title='Sharpe'),
        opacity=0.5
    )
))

# Efficient frontier
fig.add_trace(go.Scatter(
    x=np.array(frontier_vol)*100,
    y=np.array(frontier_ret)*100,
    mode='lines',
    name='Efficient Frontier',
    line=dict(color='black', width=3)
))

# Optimal portfolios
fig.add_trace(go.Scatter(
    x=[ms_vol*100, mv_vol*100],
    y=[ms_ret*100, mv_ret*100],
    mode='markers',
    name='Optimal Portfolios',
    marker=dict(size=20, color=['green', 'red'], symbol='star',
                line=dict(width=2, color='white'))
))

fig.update_layout(
    title='Portfolio Optimization: Monte Carlo + Efficient Frontier',
    xaxis_title='Volatility (%)',
    yaxis_title='Expected Return (%)',
    height=700
)

fig.show()

## 📝 Summary & Export

In [22]:
# Save optimal weights
optimal_df = pd.DataFrame({
    'Ticker': tickers,
    'Max_Sharpe_Weight': max_sharpe.x,
    'Min_Vol_Weight': min_vol.x
})

optimal_df = optimal_df[
    (optimal_df['Max_Sharpe_Weight'] > 0.001) | 
    (optimal_df['Min_Vol_Weight'] > 0.001)
].sort_values('Max_Sharpe_Weight', ascending=False)

optimal_df.to_csv('bist100_optimal_weights.csv', index=False)
print('✓ Saved: bist100_optimal_weights.csv')

optimal_df.head(20)

✓ Saved: bist100_optimal_weights.csv


,Ticker,Max_Sharpe_Weight,Min_Vol_Weight
86,EFORC.IS,1.500000e-01,5.992081e-02
32,ASELS.IS,1.500000e-01,1.061541e-02
14,PASEU.IS,1.500000e-01,7.330226e-02
46,HALKB.IS,1.231389e-01,1.234417e-17
58,ECILC.IS,8.217854e-02,6.033167e-03
56,GENTS.IS,7.894793e-02,1.776291e-02
66,AKSEN.IS,7.091606e-02,0.000000e+00
62,GRTHO.IS,5.935779e-02,1.896982e-02
81,GENIL.IS,3.594760e-02,3.576529e-02
64,ENKAI.IS,3.366400e-02,5.146383e-03


## ✅ Optimization Complete!

### Key Results:
- **Maximum Sharpe Ratio Portfolio**: Optimized for risk-adjusted returns
- **Minimum Volatility Portfolio**: Lowest risk exposure
- **Efficient Frontier**: All optimal risk-return combinations
- **Monte Carlo**: 10,000 random portfolios for comparison

### Next Steps:
1. Review optimal weights CSV file
2. Implement chosen portfolio strategy
3. Monitor performance over time
4. Rebalance periodically